# Game sessions

## Preparation

In [6]:
%run "../Utilities/Preparation.ipynb"
%run "../Static data/Google form correct answers.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,

## Functions

In [7]:
def getAllSessions( df, dropna ):
    result = df.loc[:, renamedRelevantColumns]
    result = result[result['type']=='start']
    result = result.drop('type', 1)
    if dropna:
        result = result.dropna(how='any')
    return result

def getAllSessionsOfUser( df, userId, dropna ):
    sessionsList = getAllSessions( df, dropna )
    return getAllSessionsOfUserOptimized(sessionsList, userId)

# sessionsList is assumed to contain the dataframe of userIds and sessionsIds
def getAllSessionsOfUserOptimized (sessionsList, userId):
    return sessionsList[sessionsList['userId']==userId]

# includewithoutusers=True will count sessions that do not have any userId attached
def countSessions( df, userId, includewithoutusers):
    return getAllSessionsOfUser( df, userId, includewithoutusers )['sessionId'].nunique()
    #return getAllSessionsOfUser( df, userId, includewithoutusers ).shape[0]

# sessionsList is assumed to contain the dataframe of userIds and sessionsIds
def getSessionsCountOptimized( df, userId):
    return getAllSessionsOfUserOptimized( df, userId ).shape[0]

def getUserSessionsCounts( df ):
    allSessions = getAllSessions( df, True )
    return allSessions.groupby('userId').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [ ]:
# get datetime of first significant event
# _userId is assumed to be in RedMetrics format
def getFirstEventDate( _userId ):
    _sessions = getAllSessionsOfUser(df152, _userId, True)
    
    firstGameTime = pd.Timestamp.max.tz_localize('utc')

    for session in _sessions['sessionId']:
        timedEvents = df152[df152['sessionId']==session]
        timedEvents = timedEvents.dropna(subset=['section'])
        
        if(len(timedEvents) > 0):
            timedEvents['userTime'] = timedEvents['userTime'].map(lambda t: pd.to_datetime(t, utc=True))

            earliest = timedEvents['userTime'].min()
            if(earliest < firstGameTime):
                firstGameTime = earliest

    return firstGameTime